In [23]:
%load_ext autoreload
%autoreload 2

import pyspiel
import open_spiel.python.examples.ubc_utils
from open_spiel.python.visualizations import ubc_treeviz
from open_spiel.python.examples.ubc_utils import *
from auctions.webutils import *
from absl import logging
logging.set_verbosity(logging.INFO)
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from open_spiel.python.examples.ubc_cma import *
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator, TremblingAgentDecorator, ModalAgentDecorator


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# load runs from experiments
experiments = []
experiments += ['jun10ppo']
experiments += ['jun10outcome']
runs = []
for experiment in experiments:
    runs += Experiment.objects.get(name=experiment).equilibriumsolverrun_set.all()
print(f"Found {len(runs)} runs")


Found 90 runs


In [25]:
run = EquilibriumSolverRun.objects.get(name='jun9_jun9_4_high_speed-cfr_outcomemccfr_outcome-101')

In [26]:
game, checkpoint, policy = get_results(run)
# TODO: This is not the policy you want. You want to modify the agents directly and then make a policy from that

c = checkpoint.equilibrium_solver_run.config
cfr = c.get('solver_type') == 'cfr'

# Load the environment
env_and_policy = ppo_db_checkpoint_loader(checkpoint, cfr=cfr)



In [27]:
for player in range(game.num_players()):
    env_and_policy.agents[player] = ModalAgentDecorator(env_and_policy.agents[player])
policy = env_and_policy.make_policy()

In [28]:
# Alternatively, to load a BR

# br_player = 0
# br_name = 'feb4/lstm'
# br_agent = load_dqn_agent(BestResponse.objects.get(
#     checkpoint = checkpoint,
#     br_player = br_player,
#     name = br_name
# ))
# policy._policies[br_agent.player_id] = br_agent


In [29]:
# Alternatively, to get from CFR directly
# solver = pyspiel.CFRSolver(game)
# for i in range(1000):
#     solver.evaluate_and_update_policy()
# policy = solver.average_policy()

In [32]:
game_name = checkpoint.game.name

node_policy_decorator, edge_policy_decorator = ubc_treeviz.make_policy_decorators(policy)
gametree = ubc_treeviz.GameTree(
    game,
    node_decorator=node_policy_decorator,
    edge_decorator=edge_policy_decorator,
    group_infosets=False,
    group_terminal=False,
    group_pubsets=False, 
    target_pubset='*',
    depth_limit=20,
    state_prob_limit=0.001,
    action_prob_limit=0.01, 
    policy=policy
)

fname = f'{checkpoint}-{game_name}'
fname = fname.replace('/', '_')
outfile= f'./graphviz/{fname}.pdf'

# logging.info("Game tree:\n%s", gametree.to_string())

gametree.draw(outfile, prog='dot')
print("Game tree saved to file", outfile)


Game tree saved to file ./graphviz/jun9_jun9_4_high_speed-cfr_outcomemccfr_outcome-101 (jun10outcome) Iteration 9999999-jun9_jun9_4_high_speed.json.pdf


In [34]:
game.auction_params

AuctionParams(opening_prices=[32, 19], licenses=array([1, 2]), license_names=['Ontario', 'Quebec'], activity=[32, 19], num_products=2, increment=0.6, reveal_type_round=-1, fold_randomness=True, skip_single_chance_nodes=True, max_round=5, player_types=defaultdict(<class 'list'>, {0: [{'prob': 0.5, 'bidder': <open_spiel.python.games.clock_auction_bidders.EnumeratedValueBidder object at 0x7f2341030700>}, {'prob': 0.5, 'bidder': <open_spiel.python.games.clock_auction_bidders.EnumeratedValueBidder object at 0x7f2341030160>}], 1: [{'prob': 0.5, 'bidder': <open_spiel.python.games.clock_auction_bidders.EnumeratedValueBidder object at 0x7f2341030760>}, {'prob': 0.5, 'bidder': <open_spiel.python.games.clock_auction_bidders.EnumeratedValueBidder object at 0x7f2341030e80>}]}), all_bids=array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2]]), bid_to_index={(0, 0): 0, (0, 1): 1, (0, 2): 2, (1, 0): 3, (1, 1): 4, (1, 2): 5}, all_bids_activity=array([ 0, 19, 38, 32, 5

In [37]:
dir(gmae)

NameError: name 'gmae' is not defined